# Анализ базы данных сервиса для чтения книг (SQL)

<div class="alert alert-info"> Задача: проанализировать базу данных, которая содержит информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

<br>Для выполнения задачи проекта необходимо выполнить следующие шаги:

 - Посчитать, сколько книг вышло после 1 января 2000 года;
 - Для каждой книги посчитать количество обзоров и среднюю оценку;
 - Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
 - Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
 - Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

### 1. Посмотрим на таблицы, содержащиеся в базе данных

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### 2. Посчитаем, сколько книг вышло после 1 января 2000 года

In [6]:
query = '''
    SELECT COUNT(*) AS count_books
    FROM books
    WHERE publication_date > '2000-01-01'
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count_books
0,819


In [7]:
query = '''
    SELECT
        COUNT(*) AS total_books,
        MIN(publication_date) AS latest_publication_year,
        MAX(publication_date) AS latest_publication_year
    FROM
        books;
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,total_books,latest_publication_year,latest_publication_year
0,1000,1952-12-01,2020-03-31


<div class="alert alert-info"> Большая часть книг вышла после 1999 года - 891 книга. При этом всего в базе данных 1000 книг, вышедших с 1952 по 2020 года.

### 3. Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query = '''
    SELECT
        b.book_id,
        b.title,
        COUNT(DISTINCT r.review_id) AS review_count,
        ROUND(AVG(rt.rating), 2) AS average_rating
    FROM
        books b
    LEFT JOIN reviews r ON b.book_id = r.book_id
    LEFT JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY
        b.book_id,
        b.title
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_count,average_rating
0,1,'Salem's Lot,2,3.67
1,2,1 000 Places to See Before You Die,1,2.50
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,4,1491: New Revelations of the Americas Before C...,2,4.50
4,5,1776,4,4.00
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,997,Xenocide (Ender's Saga #3),3,3.40
997,998,Year of Wonders,4,3.20
998,999,You Suck (A Love Story #2),2,4.50


In [9]:
query = '''
    SELECT
        MAX(review_count) AS max_review_count,
        MIN(review_count) AS min_review_count,
        ROUND(AVG(review_count), 2) AS average_review,
        MAX(average_rating) AS max_average_rating,
        MIN(average_rating) AS min_average_rating,
        ROUND(AVG(average_rating), 2) AS average_rating
    FROM (
        SELECT
            b.book_id,
            b.title,
            COUNT(DISTINCT r.review_id) AS review_count,
            ROUND(AVG(rt.rating), 2) AS average_rating
        FROM
            books b
        LEFT JOIN reviews r ON b.book_id = r.book_id
        LEFT JOIN ratings rt ON b.book_id = rt.book_id
        GROUP BY
            b.book_id,
            b.title
    ) AS subquery
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,max_review_count,min_review_count,average_review,max_average_rating,min_average_rating,average_rating
0,7,0,2.79,5.0,1.5,3.9


<div class="alert alert-info"> Максимальное количество обзоров на одну книгу - 7, но есть книги и без обзоров. В среднем у каждой книги около 3 обзоров. Средний рейтинг каждой книги от 1.5 до 5, а если говорить о среднем рейтинге по всем книгам, он равен 3.9. 

### 4. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [10]:
query = '''
    SELECT
        p.publisher_id,
        p.publisher,
        COUNT(b.book_id) AS book_count
    FROM
        books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE
        b.num_pages > 50
    GROUP BY
        p.publisher_id,
        p.publisher
    ORDER BY
        book_count DESC
    LIMIT 1
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


<div class="alert alert-info"> Больше всего книг толще 50 страниц выпустило издательство Penguin Books, у них 42 выпущенные книги.

### 5. Определим автора с самой высокой средней оценкой книг, учитывая только книги с >= 50 оценками

In [11]:
query = '''
    WITH BookRatingCounts AS (
        SELECT
            b.book_id,
            b.author_id,
            COUNT(rt.rating_id) AS rating_count,
            AVG(rt.rating) AS avg_rating
        FROM
            books b
        JOIN ratings rt ON b.book_id = rt.book_id
        GROUP BY
            b.book_id,
            b.author_id
        HAVING
            COUNT(rt.rating_id) >= 50
    ),
    AuthorAverageRatings AS (
        SELECT
            brc.author_id,
            ROUND(AVG(brc.avg_rating),2) AS author_avg_rating
        FROM
            BookRatingCounts brc
        GROUP BY
            brc.author_id
    )
    SELECT
        a.author_id,
        a.author,
        aar.author_avg_rating
    FROM
        AuthorAverageRatings aar
    JOIN authors a ON aar.author_id = a.author_id
    ORDER BY
        aar.author_avg_rating DESC
    LIMIT 1
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.28


<div class="alert alert-info"> Автором с самой высокой средней оценкой книг является J.K. Rowling и иллюстратор Mary GrandPré. Скорее всего такую высокую оценку они получили в основном за серию книг Гарри Поттера, ведь она действительно является одной из самых популярных и любимых у многих читателей серией книг.

### 6. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
    WITH UserReviewCounts AS (
        SELECT
            r.username,
            COUNT(r.review_id) AS review_count
        FROM
            reviews r
        WHERE
            r.username IN (
                SELECT
                    rt.username
                FROM
                    ratings rt
                GROUP BY
                    rt.username
                HAVING
                    COUNT(rt.rating_id) > 48
            )
        GROUP BY
            r.username
    )
    SELECT
        AVG(urc.review_count) AS avg_review_count
    FROM
        UserReviewCounts urc
    '''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review_count
0,24.0


<div class="alert alert-info"> В среднем пользователи, которые поставили больше 48 оценок, оставили 24 обзора на книги. То есть на каждую вторую книгу, которую они оценили, они написали обзор.